## Alexnet implementation in Tensorflow
source: https://github.com/allenai/XNOR-Net

In [6]:
import tensorflow as tf
import numpy as np

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

x = tf.placeholder(tf.float32, [None, 224*224*3])
x_image = tf.reshape(x, [-1,224,224,3])

def conv2d(x, W, b, strides=[1,1,1,1], padding=[[0,0],[0,0],[0,0],[0,0]]):
    x_padded = tf.pad(x, padding, 'CONSTANT')
    h_conv = tf.nn.conv2d(x_padded, W, strides, 'VALID') + b
    normed = tf.contrib.layers.batch_norm(h_conv, epsilon=1e-3)
    return tf.nn.relu(normed)

def max_pool(x, ksize, strides):
    return tf.nn.max_pool(x, ksize=ksize, strides=strides, padding='VALID')

W_conv1 = weight_variable([11, 11, 3, 96])
b_conv1 = bias_variable([96])
h_conv1 = conv2d(x_image, W_conv1, b_conv1, [1, 4, 4, 1], [[0,0],[2,2],[2,2],[0,0]])
h_pool1 = max_pool(h_conv1, [1,3,3,1], [1,2,2,1])

W_conv2 = weight_variable([5, 5, 96, 256])
b_conv2 = bias_variable([256])
h_conv2 = conv2d(h_pool1, W_conv2, b_conv2, [1, 1, 1, 1], [[0,0],[2,2],[2,2],[0,0]])
h_pool2 = max_pool(h_conv2, [1,3,3,1], [1,2,2,1])

W_conv3 = weight_variable([3, 3, 256, 384])
b_conv3 = bias_variable([384])
h_conv3 = conv2d(h_pool2, W_conv3, b_conv3, [1, 1, 1, 1], [[0,0],[1,1],[1,1],[0,0]])

W_conv4 = weight_variable([3, 3, 384, 384])
b_conv4 = bias_variable([384])
h_conv4 = conv2d(h_conv3, W_conv4, b_conv4, [1, 1, 1, 1], [[0,0],[1,1],[1,1],[0,0]])

W_conv5 = weight_variable([3, 3, 384, 256])
b_conv5 = bias_variable([256])
h_conv5 = conv2d(h_conv4, W_conv5, b_conv5, [1, 1, 1, 1], [[0,0],[1,1],[1,1],[0,0]])
h_pool3 = max_pool(h_conv5, [1,3,3,1], [1,2,2,1])

prob = tf.placeholder(tf.float32)
h_drop1 = tf.nn.dropout(h_pool3, prob)

W_conv6 = weight_variable([6, 6, 256, 4096])
b_conv6 = bias_variable([4096])
h_conv6 = conv2d(h_drop1, W_conv6, b_conv6, [1, 1, 1, 1])

h_drop2 = tf.nn.dropout(h_conv6, prob)

W_conv7 = weight_variable([1, 1, 4096, 4096])
b_conv7 = bias_variable([4096])
h_conv7 = conv2d(h_drop2, W_conv7, b_conv7)

nClasses = 1000
W_conv8 = weight_variable([1, 1, 4096, nClasses])
b_conv8 = bias_variable([nClasses])
y = conv2d(h_conv7, W_conv8, b_conv8)[:,0,0,:]

y_ = tf.placeholder(tf.float32, [None, nClasses])

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
train_step = tf.train.AdamOptimizer(1e-1, beta1=0.01).minimize(cross_entropy)

In [5]:
sess = tf.Session()
sess.as_default()
sess.run(tf.global_variables_initializer())
numbers = np.ones(224*224*3).reshape(1,224*224*3)
ys = np.zeros((1,1000))
ys[0][2] = 1
result = cross_entropy.eval(session=sess, feed_dict={x: numbers, prob: 0.5, y_: ys})
print(result)

6.90776
